# Predicting the Sale Price of Bulldozers using Machine Learning

>In this notebook, our goal is to create a machine learning model that can predict future sales prices of bulldozers based on previous bulldozer sales data 

## Problem Definition

>We need to create a model to predict the future sale price of a bulldozer, given its characteristics and previous examples of how much similar bulldozers have been sold for

## Data
>The data is downloaded from the Kaggle Bluebook for Bulldozers: https://www.kaggle.com/competitions/bluebook-for-bulldozers/data

>**There are three main data sets:**
>*     Train.csv is the training set, which contains data through the end of 2011.
>*     Valid.csv is the validation set, which contains data from January 1, 2012 - April 30, 2012
>*     Test.csv is the test set

>Data Dictionary can be found at: https://www.kaggle.com/c/bluebook-for-bulldozers/overview/evaluation

## Evaluation

>The evaluation metric for this competition is the RMSLE. Additional metrics will be included in the conclusion

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, r2_score

In [2]:
# Load the data
df = pd.read_csv("data/trainandvalid.csv", low_memory=False, parse_dates=["saledate"])

In [3]:
# Sort the DataFrame by saledate
df.sort_values(by=['saledate'], inplace=True, ascending=True)

In [4]:
# Make a copy of the original DataFrame
df_tmp = df.copy()

In [5]:
def preprocess_data(df):
    """
    Performs transformations on df and returns the transformed df.
    """
    # Extract date-related features
    df["saleYear"] = df.saledate.dt.year
    df["saleMonth"] = df.saledate.dt.month
    df["saleDay"] = df.saledate.dt.day
    df["saleDayOfWeek"] = df.saledate.dt.dayofweek
    df["saleDayOfYear"] = df.saledate.dt.dayofyear

    # Drop the original 'saledate' column
    df.drop("saledate", axis=1, inplace=True)

    # Fill the numeric rows with the median
    for label, content in df.items():
        if pd.api.types.is_numeric_dtype(content):
            if pd.isnull(content).sum():
                # Add a binary column indicating if the data was missing
                df[label + "_is_missing"] = pd.isnull(content)
                # Fill missing numeric values with the median
                df[label] = content.fillna(content.median())

    # Handle categorical missing data and convert categories to numbers
    for label, content in df.items():
        if not pd.api.types.is_numeric_dtype(content):
            df[label + "_is_missing"] = pd.isnull(content)
            # Add +1 to the category code because pandas encodes missing categories as -1
            df[label] = pd.Categorical(content).codes + 1

    return df

In [6]:
preprocess_data(df_tmp)

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,Undercarriage_Pad_Width_is_missing,Stick_Length_is_missing,Thumb_is_missing,Pattern_Changer_is_missing,Grouser_Type_is_missing,Backhoe_Mounting_is_missing,Blade_Type_is_missing,Travel_Controls_is_missing,Differential_Type_is_missing,Steering_Controls_is_missing
205615,1646770,9500.0,1126363,8434,132,18.0,1974,0.0,0,4593,...,True,True,True,True,True,False,False,False,True,True
274835,1821514,14000.0,1194089,10150,132,99.0,1980,0.0,0,1820,...,True,True,True,True,True,True,True,True,False,False
141296,1505138,50000.0,1473654,4139,132,99.0,1978,0.0,0,2348,...,True,True,True,True,True,False,False,False,True,True
212552,1671174,16000.0,1327630,8591,132,99.0,1980,0.0,0,1819,...,True,True,True,True,True,True,True,True,False,False
62755,1329056,22000.0,1336053,4089,132,99.0,1984,0.0,0,2119,...,True,True,True,True,True,False,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410879,6302984,16000.0,1915521,5266,149,99.0,2001,0.0,0,2101,...,True,True,True,True,True,False,False,False,True,True
412476,6324811,6000.0,1919104,19330,149,99.0,2004,0.0,0,240,...,True,True,True,True,True,True,True,True,True,True
411927,6313029,16000.0,1918416,17244,149,99.0,2004,0.0,0,627,...,True,True,True,True,True,True,True,True,True,True
407124,6266251,55000.0,509560,3357,149,99.0,1993,0.0,0,83,...,True,True,True,True,True,True,True,True,True,True


In [7]:
# Split the data into training and validation sets
df_val = df_tmp[df_tmp.saleYear == 2012]
df_train = df_tmp[df_tmp.saleYear != 2012]

# Split data into X and y
X_train, y_train = df_train.drop("SalePrice", axis=1), df_train.SalePrice
X_valid, y_valid = df_val.drop('SalePrice', axis=1), df_val.SalePrice

In [8]:
# Create evaluation functions
def rmsle(y_test, y_preds):
    return np.sqrt(mean_squared_log_error(y_test, y_preds))

def show_scores(model):
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_valid)
    scores = {
        "Training MAE": mean_absolute_error(y_train, train_preds),
        "Valid MAE": mean_absolute_error(y_valid, val_preds),
        "Training RMSLE": rmsle(y_train, train_preds),
        "Valid RMSLE": rmsle(y_valid, val_preds),
        "Training R^2": r2_score(y_train, train_preds),
        "Valid R^2": r2_score(y_valid, val_preds)
    }
    return scores

In [9]:
# Build an initial model
model = RandomForestRegressor(n_jobs=-1, random_state=42)
model.fit(X_train, y_train)
show_scores(model)

{'Training MAE': 1578.2015208725463,
 'Valid MAE': 6120.959637950402,
 'Training RMSLE': 0.08426351714047395,
 'Valid RMSLE': 0.25557446689440955,
 'Training R^2': 0.9875150955953674,
 'Valid R^2': 0.8727175969771312}

## Hyperparameter Tuning

In [10]:
# Define hyperparameters for RandomizedSearchCV
rf_grid = {
    "n_estimators": np.arange(10, 100, 10),
    "max_depth": [None, 3, 5, 10],
    "min_samples_split": np.arange(2, 20, 2),
    "min_samples_leaf": np.arange(1, 20, 2),
    "max_samples": [10000]
}

In [11]:
# Perform RandomizedSearchCV
rs_model = RandomizedSearchCV(
    model,
    param_distributions=rf_grid,
    n_iter=2,
    cv=5,
    verbose=True,
    random_state = 42
)

In [12]:
rs_model.fit(X_train, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


RandomizedSearchCV(cv=5,
                   estimator=RandomForestRegressor(n_jobs=-1, random_state=42),
                   n_iter=2,
                   param_distributions={'max_depth': [None, 3, 5, 10],
                                        'max_samples': [10000],
                                        'min_samples_leaf': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19]),
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18]),
                                        'n_estimators': array([10, 20, 30, 40, 50, 60, 70, 80, 90])},
                   random_state=42, verbose=True)

In [13]:
# Display best hyperparameters
rs_model.best_params_

{'n_estimators': 70,
 'min_samples_split': 4,
 'min_samples_leaf': 19,
 'max_samples': 10000,
 'max_depth': 10}

In [14]:
# Evaluate the RandomizedSearch model
show_scores(rs_model)

{'Training MAE': 6821.150352964018,
 'Valid MAE': 8310.4555231045,
 'Training RMSLE': 0.3032463960201095,
 'Valid RMSLE': 0.32973706700349353,
 'Training R^2': 0.7952976119364494,
 'Valid R^2': 0.7689681479604826}

In [15]:
# Train a model with the best hyperparameters

best_hyperparameters = rs_model.best_params_

ideal_model = RandomForestRegressor(
    n_estimators=best_hyperparameters['n_estimators'],
    min_samples_leaf=best_hyperparameters['min_samples_leaf'],
    min_samples_split=best_hyperparameters['min_samples_split'],
    max_features=1.0,
    n_jobs=-1,
    max_samples=None,
    random_state=42
)

In [16]:
ideal_model.fit(X_train, y_train)

RandomForestRegressor(min_samples_leaf=19, min_samples_split=4, n_estimators=70,
                      n_jobs=-1, random_state=42)

In [17]:
# Evaluate the ideal model
show_scores(ideal_model)

{'Training MAE': 4302.1276429165355,
 'Valid MAE': 6243.6630971757295,
 'Training RMSLE': 0.20370407588971506,
 'Valid RMSLE': 0.2557312923557643,
 'Training R^2': 0.9108342722582703,
 'Valid R^2': 0.8635763224331541}

In [18]:
# Load and preprocess the test data

df_test = pd.read_csv('data/test.csv', parse_dates=['saledate'])
df_test = preprocess_data(df_test)

In [19]:
# Ensure 'auctioneerID_is_missing' column is present

df_test["auctioneerID_is_missing"] = False
df_test.head()

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,fiBaseModel,...,Stick_Length_is_missing,Thumb_is_missing,Pattern_Changer_is_missing,Grouser_Type_is_missing,Backhoe_Mounting_is_missing,Blade_Type_is_missing,Travel_Controls_is_missing,Differential_Type_is_missing,Steering_Controls_is_missing,auctioneerID_is_missing
0,1227829,1006309,3168,121,3,1999,3688.0,2,499,180,...,True,True,True,True,True,True,True,True,True,False
1,1227844,1022817,7271,121,3,1000,28555.0,1,831,292,...,True,True,True,True,True,True,True,False,False,False
2,1227847,1031560,22805,121,3,2004,6038.0,3,1177,404,...,False,False,False,False,True,True,True,True,True,False
3,1227848,56204,1269,121,3,2006,8940.0,1,287,113,...,False,False,False,False,True,True,True,True,True,False
4,1227863,1053887,22312,121,3,2005,2286.0,2,566,196,...,True,True,True,True,False,False,False,True,True,False


In [20]:
# Reorder the columns in df_test to match the order in X_train
df_test = df_test[X_train.columns]

In [21]:
# Make predictions on the test data
test_preds = ideal_model.predict(df_test)

In [22]:
# Prepare the predictions for submission
df_preds = pd.DataFrame()
df_preds['SalesID'] = df_test['SalesID']
df_preds['SalesPrice'] = test_preds

In [23]:
# Save predictions to a CSV file
df_preds.to_csv('data/test_predictions.csv', index=False)

# Conclusion



The model achieved the best performance with hyperparameters found using RandomizedSearchCV. At the time of last execution of this notebook, the best hyperparameters were:

>`{'n_estimators': 70,
 'min_samples_split': 4,
 'min_samples_leaf': 19,
 'max_samples': 10000,
 'max_depth': 10}`

Results of model using hyperparameters listed above:

>`{'Training MAE': 4302.13,
 'Valid MAE': 6243.66,
 'Training RMSLE': 0.204,
 'Valid RMSLE': 0.256,
 'Training R^2': 0.911,
 'Valid R^2': 0.864}`
 
MAE (Mean Absolute Error):

>the model has a Training MAE of approximately \\$4,302.13, which means, on average, the predicted sale prices are off by around \\$4,302.13 when compared to the actual sale prices on the training data.
>
>The model has a validation MAE of \\$6,243.66 which indicates, on average, the predicted sale prices are off by about \\$6,243.66 compared to the actual sale prices on the validation data.


RMSLE (Root Mean Squared Log Error):


>The model has a Training RMSLE value of approximately 0.204 suggests that the model's predicted sale prices deviate from the actual sale prices by a factor of about 20.4% on the training data.
>
>
>The model has a Validation RMSLE of  0.256 implies that, on average, the model's predictions are off by a factor of about 25.6% compared to the actual sale prices on the validation data.


R^2 (R-squared):

>The model has a Training R^2 of 0.911 means that the model can explain approximately 91.1% of the variance in the training data. 
>
>In other words, the model captures about 95.9% of the variability in sale prices in the training dataset.
Validation R^2 (R-squared):
>
>
>The model has a A Validation R^2 of 0.0.864 indicates that the model explains about 86.4% of the variance in the validation data, suggesting a reasonably good ability to generalize to new, unseen data.